# Small Temporal Density Variations


The following code is used to visualise density varitions from photographs of a moving ridge in the lab. The code firstly takes a background photograph, finds the density profile then compares said profile to photographs taken of the moving ridge.

In [1]:
%matplotlib qt
#notebook backend important
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os.path
import cv2
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askopenfilenames
import matplotlib.animation as animation
from scipy import signal 
import cmocean as cmo
import time

In [2]:
#A simple function used when clicking of photos to determine where in an array the click is
def index_find(depths,click):
    lst = depths<click
    for i,v in enumerate(lst):
        if v==True:
            return i

## Loading the data from the experiments 

In [3]:
excel_path = askopenfilename() #path to excel doc with data
data = pd.read_excel(excel_path)

In [5]:
%matplotlib qt

run_num = int(input('Run number:'))
print('Data read')
#getting data from excel doc
rho_bottom = data.iloc[run_num-1, 6]
rho_top = data.iloc[run_num-1, 7]
depth = data.iloc[run_num-1, 2]



Run number:5
Data read


## Analysing the background image

In [6]:
#now choose background image, this image should not contain topography

background_path = askopenfilename() 
b_image = cv2.imread(background_path,0)

Now we choose the area of photograph we want to average over to create a background stratification profile. The top of the region should be the free surface and the bottom should be the base of the tank.

In [7]:
plt.figure()
plt.title('Choose Area to Average over')
plt.axis('off')
plt.imshow(b_image, cmap='gist_gray')
print('Select the top left then the bottom right' )
density_locations = plt.ginput(2)
print('Locations recorded')

zbot_0=int(np.round(density_locations[0][1]))
ztop_0=int(np.round(density_locations[1][1]))
xleft=int(np.round(density_locations[0][0]))
xright=int(np.round(density_locations[1][0]))

back_crop= b_image[zbot_0:ztop_0,xleft:xright]


#cropping and determining the background profile

log_back = np.log(back_crop)

#getting rid of unwated inf_values
log_back[np.isinf(log_back)]=np.nan

mean_ratio_profile = np.mean(log_back,1)
intensity = mean_ratio_profile
depth_array = -np.linspace(0,depth,len(mean_ratio_profile))

Select the top left then the bottom right
Locations recorded


### Fixing up the background absorbtion profile

The top and bottom of the profile need to be fixed up

In [8]:
# fixing up the top of the profile, making it have a constant value
splintensity=np.copy(intensity)

#determining the length of picture
length_ratio = depth/splintensity.shape[0]

plt.figure()
plt.plot(intensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Select Top Section of Absorption Profile')
print('Choose bottom of top section')
topsection = plt.ginput(1)[0][1]
print('Location recorded')
ztop=index_find(depth_array,topsection) 
splintensity[:ztop]=splintensity[ztop]

Choose bottom of top section
Location recorded


In [9]:
#fixing up the bottom of the profile, giving it a linear function
#can skip this step if camera is not taking in the bottom of the tank


plt.figure()
plt.plot(splintensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Select Bottom Section of Absorption Profile')
print('Choose top then bottom')
bottomsection = plt.ginput(2)
print('Locations recorded')

#finding equation of line
slope = (bottomsection[0][1]-bottomsection[1][1])/(bottomsection[0][0]-bottomsection[1][0])

zbot=index_find(depth_array, bottomsection[0][1]) 

splintensity[zbot:]=slope*np.linspace(0,bottomsection[0][1], len(splintensity)-zbot)+splintensity[zbot]

Choose top then bottom
Locations recorded


In [9]:
#plot that shows the raw vs the 'fixed' absorbtion profile

plt.figure()
plt.title('Absorbtion Profile')
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.plot(intensity,depth_array, 'b', label='Raw')
plt.plot(splintensity,depth_array, 'r',label='Updated')
plt.legend()

In [10]:
#determining the location of density samples top then bottom
plt.figure()
plt.title('Density Sample Locations')
plt.ylabel('z (m)')
plt.xlabel('x (m)')
plt.imshow(back_crop, cmap='gist_gray', extent=[0,length_ratio*back_crop.shape[1],-depth,0])
print('Position of density sample close to top, then close to bottom' )
density_locations = plt.ginput(2)
print('Locations recorded')

zbot_rho=index_find(depth_array,density_locations[1][1])
ztop_rho=index_find(depth_array,density_locations[0][1])

beta = (rho_bottom-rho_top)/(splintensity[zbot_rho]-splintensity[ztop_rho])
splintensity_bottom_ref = splintensity[zbot_rho]

rho = rho_bottom+np.float64(beta*(splintensity-splintensity_bottom_ref))

Position of density sample close to top, then close to bottom
Locations recorded


In [11]:
#plot of background density
plt.figure()
plt.plot(rho, depth_array)
plt.xlabel('rho (kg/m^3)')
plt.ylabel('z (m)')
plt.title('Background Density')

Text(0.5, 1.0, 'Background Density')

## Foreground Images
Choose the rapid foreground images you want in order to make a video of the moving ridge in density space


In [12]:
#now choose the pictures you want to look over, also creates a folder to save the video
foreground_path = askopenfilenames()
no_images = len(foreground_path)
os.mkdir('{}/densityfields2'.format(os.path.dirname(foreground_path[0])))

In [13]:
%matplotlib inline
print('{} Images to analyse!'.format(no_images))
#inline backend used to stop hundreds of plots occuring, very important!!!!!!!

460 Images to analyse!


Below is code used to generation video of ridge and determing if density anomaly or absolute density is wanted

In [13]:
what_field = int(input('What field? 1 for anomaly or 2 for absolute: '))

What field? 1 for anomaly or 2 for absolute: 1


In [16]:
density_array = np.zeros((no_images,1562,2988))

In [19]:
ims=[]
start = time.time()
fig = plt.figure(figsize=(10,5))

for i in range(no_images):

    f_image = cv2.imread(foreground_path[i],0)
    f_image_crop=f_image[zbot_0:ztop_0,:]
    log = np.log(f_image_crop)

    #getting rid of unwated inf_values
    log[np.isinf(log)]=np.nan
    
    density = rho_bottom+np.float64(beta*(log-splintensity_bottom_ref))
    
    if what_field == 1:
        
        title = 'Run {}- Density Anomaly'.format(run_num)
        save_name = 'run_{}_anomaly'.format(run_num)
        
        cmap = cmo.cm.balance
        vmin=-2.5
        vmax=-vmin
        
        den=density[:,50:-50]-rho[:,None]
        den[den>4]=np.nan
        density_filt=signal.medfilt2d(den[50:,:],3)
        
        plt_depth=depth
        tank_length=length_ratio*density_filt.shape[1]
    
    if what_field == 2:
        
        title = 'Run {}- Absolute Density'.format(run_num)
        save_name = 'run_{}_absolute'.format(run_num)
        
        cut=600
        plt_depth=depth/718*(718-cut)
            
        den=density[:,50:-50]
        den[den>rho_bottom+2]=np.nan
        density_filt=cv2.medianBlur(den[:cut,:],3)
    
        tank_length=length_ratio*density_filt.shape[1]
    
        cmap = cmo.cm.dense
        vmin=rho_top
        vmax=rho_top+4
        
        
    plt.xlabel('Length (m)')
    plt.ylabel('Depth (m)')
    plt.title(title, fontsize=20)


    im=plt.imshow(density_filt, cmap=cmap, animated=True, vmin=vmin,vmax=vmax, extent=[0,tank_length,-plt_depth,0])
    ims.append([im])
    
    density_array[i]=den
    
    if i % 25 == 0:
        print('{} of {} Images Done!'.format(i,no_images))
        
print(time.time()-start)        
ani = animation.ArtistAnimation(fig, ims, interval=125, blit=True,
                                repeat_delay=1000)

#colorbar

ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar=plt.colorbar(im, cax=cax)
cbar.set_label('Density (kg/m3)', rotation=90)


print('Saving!')

writer = animation.writers['ffmpeg']
ani.save('{}/densityfields2/2{}.mp4'.format(os.path.dirname(foreground_path[0]),save_name), dpi=250)
np.savez('{}/densityfields2/data'.format(os.path.dirname(foreground_path[0])), density_array=density_array, rho=rho,depth=depth,depth_array=depth_array, tank_length=tank_length,plt_depth=plt_depth)


C:\Users\u6201343\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':
C:\Users\u6201343\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


0 of 460 Images Done!
25 of 460 Images Done!
50 of 460 Images Done!
75 of 460 Images Done!
100 of 460 Images Done!
125 of 460 Images Done!
150 of 460 Images Done!
175 of 460 Images Done!
200 of 460 Images Done!
225 of 460 Images Done!
250 of 460 Images Done!
275 of 460 Images Done!
300 of 460 Images Done!
325 of 460 Images Done!
350 of 460 Images Done!
375 of 460 Images Done!
400 of 460 Images Done!
425 of 460 Images Done!
450 of 460 Images Done!
426.12710785865784
Saving!


## Finding Velocities 

Plan to use:

$\frac{D b}{D t}+N^2(z)w\approx 0,$

 to determine $w$. This yields:
 
 $w=-\frac{1}{N^2}\frac{D b}{D t},$

and then use $\nabla \mathbf{u}=0$ to determine $u$. 

This method does assume there is negligible mixing and is not yet fully implimented. Going to using background stratification to determine N, looks like there is two distinct regions- the thermocline where $N^2=0$ and below the thermocline where $N$ is stable $(N^2>0)$.




In [97]:
#loading in data
data_path=askopenfilename() 
data=np.load(data_path)

rho=data['rho']
depth=data['depth']
depth_array=data['depth_array']
density_array=data['density_array']
tank_length=data['tank_length']
plt_depth=data['plt_depth']


In [98]:
#Determining thermocline boundary, from background density profile
plt.figure()
plt.plot(rho,depth_array)
plt.xlabel('Density (kg/m3)')
plt.ylabel('z (m)')
plt.title('Select Bottom Section of Thermocline')
bottom_thermocline = plt.ginput(1)
print('Location recorded')

C:\Users\u6201343\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Location recorded


In [99]:
N=np.zeros(len(rho))
g = 9.81
bottom=index_find(depth_array,bottom_thermocline[0][1])
rho_0=rho[bottom]
rho_max=rho[-1]
N[:bottom]=0
N[bottom:]=np.sqrt(g/rho_0*(rho_max-rho_0)/(depth+bottom_thermocline[0][1]))

In [55]:
density_array.shape[1]

600

In [100]:
#now we want to find Db/dt
w=np.zeros((density_array.shape[0]-1,density_array.shape[1],density_array.shape[2]))
delta_t=0.125
sf=1
scaling = -sf/N**2
for i in range(w.shape[0]):
    diff = (density_array[i+1]-density_array[i])/delta_t
    w[i] = diff*scaling[:,None]
    

C:\Users\u6201343\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
C:\Users\u6201343\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


In [122]:
plt.figure()
plt.imshow(cv2.medianBlur(w[5],4)[:600,:],vmin=-10,vmax=10,cmap=cmo.cm.balance)
plt.colorbar()

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\median_blur.dispatch.cpp:283: error: (-215:Assertion failed) (ksize % 2 == 1) && (_src0.dims() <= 2 ) in function 'cv::medianBlur'


In [69]:
sf=1
scaling = -sf/N**2

C:\Users\u6201343\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [109]:
domain = np.identity(3)

In [71]:
N

array([0.        , 0.        , 0.        , ..., 0.84104086, 0.84104086,
       0.84104086])

In [68]:
N.shape

(1569,)

In [20]:
1+1

2